### `PART 4` - Decompounded Options

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate

import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
swap_data = pd.read_csv("swap.csv")
swap_data

,Unnamed: 0,start,tenor,end,swap_rates
0,0,1,1,2,0.032007
1,1,1,2,3,0.033259
2,2,1,3,4,0.034011
3,3,1,5,6,0.035255
4,4,1,10,11,0.038428
5,5,5,1,6,0.039274
6,6,5,2,7,0.040075
7,7,5,3,8,0.040072
8,8,5,5,10,0.041093
9,9,5,10,15,0.043634


In [4]:
OIS_DF = pd.read_csv("OIS_DF.csv")
OIS_DF

,Unnamed: 0,f_list,OIS_discount_list
0,1,0.003494,0.997009
1,2,0.003495,0.993531
2,3,0.003545,0.990015
3,4,0.003946,0.986117
4,5,0.003996,0.982184
5,6,0.005003,0.977283
6,7,0.005003,0.972406
7,8,0.005680,0.966898
8,9,0.005680,0.961422
9,10,0.005680,0.955977


#### Question 1

For static replication of any constant maturity swap (CMS) payoff $g(F)$, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

Let $m$ denote the payment frequenc ($m=2$ for semi-annual payment frequency), and let $N$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\mbox{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.


In [5]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


For CMS rate payment, since $g(F)= CMS10y^{1/p} - 0.04^{1/q}$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &=  (F)^{\frac{1}{p}} - 0.04^{\frac{1}{q}}\\
g'(K) &= \frac{1}{p}(F)^{\frac{1}{p}-1} \\
g''(K) &= (\frac{1}{p})(\frac{1}{p}-1)(F)^{\frac{1}{p}-2}
\end{split}
\end{equation*}

In [6]:
p,q = 4,2

In [7]:
def g_0(K,p,q):
    return K**(1/p) - 0.04**(1/q)

def g_1(K,p,q):
    return (1/p)*K**((1/p)-1)

def g_2(K,p,q):
    return (1/p)*((1/p)-1)*K**((1/p)-2)

The function $h(K) = g(K)/IRR(K)$ now simplies into:  
  
  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}


In [8]:
def h_0(K, m, N, p, q):
    # implementation of h(K)
    value = g_0(K, p, q) / IRR_0(K, m, N)
    return value

def h_1(K, m, N, p, q):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K, p, q) - g_0(K, p, q)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N, p, q):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K, p, q) - IRR_2(K, m, N)*g_0(K, p, q) - 2.0*IRR_1(K, m, N)*g_1(K, p, q))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K, p, q)/IRR_0(K, m, N)**3)
    return secondDerivative

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [9]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [10]:
alpha,beta,rho, nu =0.173777, 0.9, -0.419190, 0.527253 # Calibrated parameters for the 5x10 swaption

In [11]:
#Black formula to value the receiver and payer swwaps

def Black76_Call(F, K, r, sigma, T):
    d1 = (np.log(F/K) + 0.5*sigma**2 * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return (F*norm.cdf(d1) - K*norm.cdf(d2))

def Black76_Put(F, K, r, sigma, T):
    d1 = (np.log(F/K) + 0.5*sigma**2 * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return (K*norm.cdf(-d2) - F*norm.cdf(-d1))

In [12]:
F = swap_data.loc[9, "swap_rates"]         # 5x10 forward rate
D = OIS_DF.loc[5, "OIS_discount_list"]     # 5 year OIS discount factor

In [32]:
print("The 5x10 forqward rate is given by %.5f." %F)
print("The 5 year OIS discount factor is given by %.5f." %D)

The 5x10 forqward rate is given by 0.04363.
The 5 year OIS discount factor is given by 0.97728.


In [14]:
m = 2  # Semi-annual payment
n = 10 
T = 5
r = 0



\begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}


In [15]:
V_rec = integrate.quad(lambda x: h_2(x,m,n,p,q)* D* IRR_0(x,m,n) * Black76_Put(F, x, r, 
                                                                               SABR(F, x, T, alpha, beta, rho, nu),T),
                                                0,F
                      )

V_pay = integrate.quad(lambda x: h_2(x,m,n,p,q)* D* IRR_0(x,m,n)* Black76_Call(F, x, r, 
                                                                               SABR(F, x, T, alpha, beta, rho, nu),T),
                                                F,1000
                      )


In [16]:
payoff1 = D*g_0(F,p,q) + V_rec[0] + V_pay[0]
print("The PV of the decompounded option is %.3f." %payoff1)

The PV of the decompounded option is 0.219.


#### Question 2

The option described in part 2 is similar to a caplet and can be replicated using the formula below.

  \begin{equation*}
    \begin{split}
      V_0 &= h'(L)[V^{pay}(L)] + 
      \int_L^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

To find the lower limit of the integral, we need 

\begin{equation*}
(F)^{\frac{1}{p}} > 0.04^{\frac{1}{q}}\\
\end{equation*}

Therefore the lower limit L is given by

\begin{equation*}
L = 0.04^\frac{p}{q}
\end{equation*}

In [17]:
L = 0.04**(p/q)

In [18]:
term_1 = h_1(L,m,n,p,q)* D* IRR_0(L,m,n)* Black76_Call(F, L, r, 
                                                       SABR(F, L, T, alpha, beta, rho, nu),T)

term_2 = integrate.quad(lambda x: h_2(x,m,n,p,q)* D* IRR_0(L,m,n)*Black76_Call(F, x, r, 
                                                                               SABR(F, x, T, alpha, beta, rho, nu),T),
                                                L,1000
                      )

In [19]:
payoff2 = term_1 + term_2[0]
print("The PV of the decompounded option is %.3f." %payoff2)

The PV of the decompounded option is 0.304.


`END`